# 关于计算图与自动微分的一些小心得

看了有关博客与[视频（17：26）](https://www.bilibili.com/video/BV1jK4y1g7xh?from=search&seid=204331998484761480),*算是了解了计算图和pytorch的基本微分求导机制，不得不说，之前一直没有注意看视频的重要性，都是自己在那里找纸质资料自己琢磨，实际上效率不一定会更高，有的时候就需要多样化的学习方式，赶快回学校吧，还可以与点团队，与各位大佬们交流一下*

## 导入必要的库

In [21]:
import torch as t
import numpy as np

首先我们先创建两个张量：

In [22]:
a = t.tensor(2., requires_grad=True)
b = t.tensor(4., requires_grad=True)

In [23]:
c = a + b  # c = a.add(b)

由于c是两者相加， 我们可以查看一下c的grad_fn，我们也可以看出a\b都没有grad_fn属性，只有c有，这个属性是用来存储保存之前的函数计算关系的，方便反向传播时候回溯，这个和计算图的那个东西是非常像的，~~还好我现在还对计算图有些印象， 当时就简单看了一眼长什么样子~~

In [24]:
c.grad_fn, a.grad_fn, b.grad_fn

(<AddBackward0 at 0x209120ef280>, None, None)

我们接下来接着看一下如果继续给c平方一下

In [25]:
d = c ** 2
d.grad_fn

可以看出，d的grad_fn变为pow，幂次操作，由以上关系我们可以看出，实际上我们只要设置初始的张量是可微的，那么后来由该张量得到的任意张量都是可微的，并且每一个张量都存储了之前的计算信息，保存了相关的计算函数关系这就是所谓的回溯机制而根据这个回溯机制，我们可以清楚的掌握每一步张量的计算并据此绘制**计算图**  
  
  [![](https://z3.ax1x.com/2021/08/08/fQ2pKx.png)](https://imgtu.com/i/fQ2pKx)

图的每一个结点代表每一个可微分的张量，而边代表函数运算，张量计算图就是有向无环图
    
    *当然结点也有好几种，对于不同节点的操作也是不完全一样的*：
  
  [![fQ2Us0.png](https://z3.ax1x.com/2021/08/08/fQ2Us0.png)](https://imgtu.com/i/fQ2Us0)
  例如

In [26]:
c.is_leaf, a.is_leaf, b.is_leaf

(False, True, True)

注意这里我们求a的梯度会发现a的梯度并没有计算，没有输出，这是因为我们并没有进行反向传播，仅仅前向传播了一下。  
为了节省运算资源，我们只有在指定需要反向传播是才会计算叶子结点的梯度  
*反向传播可以理解为在此前记录存储的函数关系的基础上，反向计算传播函数关系进而求得梯度*

In [27]:
a.grad

In [28]:
c

tensor(6., grad_fn=<AddBackward0>)

我们可以先看一下c的grad_fn属性

In [29]:
c.grad_fn

接着我们可以对c进行反向传播

In [30]:
c.backward()

接着我们便可以查看a的梯度值

In [31]:
a.grad

tensor(1.)

我们选择对d查看它的grad_fn并进行反向传播

In [32]:
print(d.grad_fn)
d.backward()

查看a的梯度

In [33]:
a.grad

tensor(13.)

注意一下为什么这里的结果是13而并非12，因为我们的Pytorch反向传播他**不会自动给我们进行梯度清零操作，他计算的是累计梯度，所以需要我们手动清零**

注意，默认情况下一张计算图只能执行一次反向传播，如果执行多次会报错， 并且实际上，也只有叶子结点才有grad

另外，实际上计算之后，中间节点的梯度并不会保存，只有叶子节点的梯度会被保存